# Tool Use Example

Example usage of `tool use` for converse method from Bedrock Runtime Python SDK

#### Initialize client

In [1]:
import json
import boto3
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
client = boto3.client("bedrock-runtime")

#### Define tool function

In [2]:
class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass

def get_top_song(call_sign):
    song = ""
    artist = ""
    if call_sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise StationNotFoundError(f"Station {call_sign} not found.")

    return song, artist

#### Define inputs

In [6]:
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Initial message from the user
messages = [{
    "role": "user",
    "content": [{"text": "What is the most popular song on WZPZ?"}]
}]

tool_config = {
    "tools": [{
        "toolSpec": {
            "name": "top_song",
            "description": "Get the most popular song played on a radio station.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "sign": {
                            "type": "string",
                            "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ, and WKRP."
                        }
                    },
                    "required": ["sign"]
                }
            }
        }
    }]
}

#### First message and response

In [7]:
response = client.converse(
    modelId=model_id,
    messages=messages,
    toolConfig=tool_config
)

output_message = response['output']['message']
stop_reason = response['stopReason']
messages.append(output_message)

messages

[{'role': 'user',
  'content': [{'text': 'What is the most popular song on WZPZ?'}]},
 {'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'tooluse_RE9pgLYKRMaviVRacRHTuA',
     'name': 'top_song',
     'input': {'sign': 'WZPZ'}}}]}]

#### Use the tool when the model requests it

# NEEDS BUG FIXING: This is wrong since the model output might contain multiple tool use requests

# MISSING: BREAK THE NEXT SECTION INTO: 'Call the tool', 'Send tool result'

In [ ]:
if stop_reason == 'tool_use':
    content = response['output']['message']['content']
    
    for part in content:
        if 'toolUse' in part:
            tool = part['toolUse']

            if tool['name'] == 'top_song':
                tool_result = {}
                
                try:
                    song, artist = get_top_song(tool['input']['sign'])
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"json": {"song": song, "artist": artist}}]
                    }
                    
                except StationNotFoundError as err:
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"text":  err.args[0]}],
                        "status": 'error'
                    }

                message = {
                    "role": "user",
                    "content": [{
                        "toolResult": tool_result
                    }]
                }
                
                messages.append(message)

                # Send the tool result to the model.
                response = client.converse(
                    modelId=model_id,
                    messages=messages,
                    toolConfig=tool_config
                )
                output_message = response['output']['message']

# print the final response from the model.
for content in output_message['content']:
    print(json.dumps(content, indent=4))

{
    "text": "The most popular song currently playing on radio station WZPZ is \"Elemental Hotel\" by the artist 8 Storey Hike."
}

Finished generating text with model anthropic.claude-3-haiku-20240307-v1:0.


---
### Multiple tool use requests in a single response from the model

Yes, the Bedrock Converse method with tool use can return a response message containing multiple toolUse objects.

1. Multiple Tool Requests: The model can request the use of multiple tools in a single response if it determines that multiple tools are necessary to generate a comprehensive answer.

2. Sequential Processing: When multiple tool requests are made, they are typically processed sequentially. You would need to handle each tool request one at a time, providing the results back to the model before it generates its final response.

3. Response Structure: The response message will contain separate toolUse objects for each tool request. Each toolUse object will include details such as the tool name and input parameters required for that specific tool.

4. Handling Multiple Tools: Your application should be prepared to:
   - Identify multiple toolUse objects in the response
   - Execute each requested tool
   - Compile the results from all tools
   - Send all tool results back to the model in a new Converse request

5. Tool Result Message: When sending the results back, you can include multiple toolResult content blocks in your message, each corresponding to a different tool that was used.

6. Final Response: After processing all tool results, the model will generate a final response that incorporates information from all the tools used.

7. Implementation Considerations: 
   - Ensure your code can handle variable numbers of tool requests
   - Implement error handling for cases where a tool might fail
   - Consider setting appropriate timeouts for tool executions

8. Best Practices:
   - Test your implementation with various scenarios, including multiple tool requests
   - Monitor and log tool usage for debugging and optimization
   - Regularly review and update your tool definitions to ensure they remain relevant and secure

Remember to always follow AWS best practices for security and compliance when implementing tool use with the Bedrock Converse API. For the most up-to-date and detailed information on implementing multiple tool use, please refer to the official AWS Bedrock documentation.
Sources
[1] [Call a tool with the Converse API - Amazon Bedrock] (https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-inference-call.html)
[3] [Returning tool results - Amazon Nova] (https://docs.aws.amazon.com/nova/latest/userguide/tool-use-results.html)
[6] [Community | Intro to Tool Use with the Amazon Bedrock Converse API] (https://community.aws/content/2hW5367isgQOkkXLYjp4JB3Pe16/intro-to-tool-use-with-the-amazon-bedrock-converse-api)